In [32]:
import dash
from dash import dcc, html, Input, Output, State, dash_table
import plotly.graph_objs as go
import pandas as pd
from datetime import datetime, timedelta
import os

app = dash.Dash(__name__)

data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

data_file = os.path.join(data_dir, 'baby_growth_data.csv')
if os.path.exists(data_file):
    df = pd.read_csv(data_file)
    df['Date'] = pd.to_datetime(df['Date'])
else:
    df = pd.DataFrame(columns=['Date', 'Age_Days', 'Weight_kg'])

# Load WHO data (you'll need to download this from the WHO website)
who_data = pd.read_excel('data/tab_wfa_girls_p_0_13.xlsx')

# Process the data to get the percentiles you need
percentiles = {
    '5th': who_data['P5'].tolist(),
    '10th': who_data['P10'].tolist(),
    '50th': who_data['P50'].tolist(),
    '90th': who_data['P90'].tolist(),
    '95th': who_data['P95'].tolist(),
}

growth_curves = percentiles

app.layout = html.Div([
    html.H1("Baby Growth Tracker for Preterm Female Infants (UK-WHO Standards)"),
    
    html.Div([
        html.Label("Actual Date of Birth:"),
        dcc.DatePickerSingle(id='dob-picker', date=datetime.now().date() - timedelta(days=14)),
        
        html.Label("Expected Date of Birth:"),
        dcc.DatePickerSingle(id='expected-dob-picker', date=datetime.now().date()),
        
        html.Label("Date of Measurement:"),
        dcc.DatePickerSingle(id='date-picker', date=datetime.now().date()),
        
        html.Label("Weight (kg):"),
        dcc.Input(id='weight-input', type='number', placeholder='Enter weight in kg'),
        
        html.Button('Add Record', id='add-button', n_clicks=0)
    ]),
    
    html.Div(id='record-added-message'),
    
    dcc.Graph(id='growth-chart'),
    
    dash_table.DataTable(
        id='data-table',
        columns=[
            {'name': 'Date', 'id': 'Date', 'type': 'datetime'},
            {'name': 'Age (Days)', 'id': 'Age_Days', 'type': 'numeric'},
            {'name': 'Weight (kg)', 'id': 'Weight_kg', 'type': 'numeric'}
        ],
        data=df.to_dict('records'),
        editable=True,
        row_deletable=True
    ),
    
    html.Button('Save Changes', id='save-button', n_clicks=0)
])

@app.callback(
    Output('record-added-message', 'children'),
    Output('growth-chart', 'figure'),
    Output('data-table', 'data'),
    Input('add-button', 'n_clicks'),
    Input('save-button', 'n_clicks'),
    Input('data-table', 'data'),
    State('dob-picker', 'date'),
    State('expected-dob-picker', 'date'),
    State('date-picker', 'date'),
    State('weight-input', 'value')
)
def update_data_and_chart(add_clicks, save_clicks, table_data, dob, expected_dob, date, weight):
    global df
    ctx = dash.callback_context
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if trigger_id == 'add-button' and dob and expected_dob and date and weight is not None:
        dob = datetime.strptime(dob, '%Y-%m-%d')
        expected_dob = datetime.strptime(expected_dob, '%Y-%m-%d')
        measurement_date = datetime.strptime(date, '%Y-%m-%d')
        age_days = (measurement_date - dob).days
        
        new_record = pd.DataFrame({'Date': [measurement_date], 'Age_Days': [age_days], 'Weight_kg': [weight]})
        df = pd.concat([df, new_record], ignore_index=True)
        df.to_csv(data_file, index=False)
        message = "Record added successfully!"
    elif trigger_id == 'save-button' or trigger_id == 'data-table':
        df = pd.DataFrame(table_data)
        df['Date'] = pd.to_datetime(df['Date'])
        df.to_csv(data_file, index=False)
        message = "Changes saved successfully!"
    else:
        message = "No changes made."
    
    fig = update_chart(dob, expected_dob)
    return message, fig, df.to_dict('records')

def update_chart(dob, expected_dob):
    fig = go.Figure()
    
    for percentile, weights in growth_curves.items():
        fig.add_trace(go.Scatter(
            x=weeks,
            y=weights,
            mode='lines',
            name=f'{percentile} Percentile',
            line=dict(color='rgba(0,0,0,0.3)', dash='dash' if percentile not in ['3rd', '97th'] else 'solid')
        ))
    
    if not df.empty and dob and expected_dob:
        dob = datetime.strptime(dob, '%Y-%m-%d')
        expected_dob = datetime.strptime(expected_dob, '%Y-%m-%d')
        weeks_early = (expected_dob - dob).days / 7
        age_weeks = (df['Age_Days'] / 7) - weeks_early
        fig.add_trace(go.Scatter(
            x=age_weeks,
            y=df['Weight_kg'],
            mode='lines+markers',
            line=dict(color='red', width=3),
            marker=dict(size=8),
            name="Baby's Growth"
        ))
    
    fig.update_layout(
        title='Baby Growth Chart for Preterm Female Infants (UK-WHO Standards)',
        xaxis_title='Age (weeks, adjusted for preterm)',
        yaxis_title='Weight (kg)',
        legend=dict(y=0.5, traceorder='reversed', font_size=16),
        hovermode='x unified'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
